# Fluvial corridor workflow

## Install GDAL
For Windows 10, install GDAL with OSGEO4W

[link to installer](https://trac.osgeo.org/osgeo4w/wiki/OSGeo4W_fr)

Choose *Express Install* then GDAL.

In your user environement variables add C:\OSGeo4W\bin to PATH.

In [1]:
# change working directory to root package python-fct
import os
os.chdir(os.path.dirname(os.getcwd()))

## Set input parameters
Create a config.ini file.

The outputs folder and name are generate  automatically.
They are define in yml files in the config/datasets folder or by tiledir attributes in config.ini section.

Activate the virtual environement in shell from the python-fct folder.

The initial DEM (RGEALTI) is splited into tiles with 10K_TILESET and 10KBIS_TILESET with this shell commands.

fct-tiles call shell FileCommand.py and TileCommand.py
Arguments :
- config.ini = path to config.ini with input path and parameters
- extract = Extract Tiles from Datasource for tiles defined in Tileset, and store as Dataset. Function TileCommand.py/extract/Tiles.py/DatasourceToTiles
- bdalti = input define in config.ini
- 10k = input tile grid define in config.ini
- dem = output file define in config/datasets/drainage.yml

Build virtual raster from the tiled previously created.

Arguments :
- config.ini = path to config.ini with input path and parameters
- buildvrt = Build GDAL Virtual Raster from tile dataset. Function TileCommand.py/vrt/tileio.py/buildvrt
- 10k = input tile grid define in config.ini
- dem = output file define in config/datasets/drainage.yml

If you have two different scales DEM, you can fill the precise one with the less precise
A faire en amont du lissage?

## Create a drainage networks
### DEM preparation
#### create configuration file

In [2]:
from fct.drainage import PrepareDEM
# load the config.ini file with relative path
PrepareDEM.config.from_file('./tutorials/dem_to_dgo/config.ini')

PrepareDEM.config gather all the input and output parameters
input are defined in config.ini
output are define in .yml files in config/datasets
PrepareDEM.config have 3 dictionnaries
- _tilesets for raster tiles
- _datasources for input files
- _datasets for output destination files with parameters

In [3]:
# uncomment below to see configuration definition and function
# help(PrepareDEM.config)

In [4]:
# uncomment code below to print all parameters from the config class
# vars(PrepareDEM.config)
# example : get all data sources from the config class
# PrepareDEM.config._datasources

#### DEM smoothing

In [5]:
# Prepare the smoothing parameter
params = PrepareDEM.SmoothingParameters()

smoothing parameters : 
- elevations = input
- output = output smoothed dem
- window = smoothing window

In [6]:
# uncomment below to print default smoothing parameter

In [7]:
# uncoment below to get input path 
# params.elevations.filename()

In [8]:
# change window size
params.window=25

In [9]:
# uncomment below to print new parameters
# params.__dict__.items()

In [10]:
# Mean filter for each tile of the tileset
for tile in PrepareDEM.config.tileset().tiles():
    PrepareDEM.MeanFilter(row=tile.row, col=tile.col, params=params)

In [11]:
# uncomment below to see an example of the the output tile path
# params.output.tilename(row=1, col=1)

Create virtual raster from tiles in shell (optional)

#### Fill sink

In [12]:
# import package and config
from fct.drainage import DepressionFill
DepressionFill.config.from_file('./tutorials/dem_to_dgo/config.ini')

# Pourquoi recréer le fichier de configuration???

In [13]:
# set parameters
params = DepressionFill.Parameters()

DepressionFill parameters :

- _elevations = input
- _hydrography = Stream network derived from cartography draped on DEM **a quoi ça sert??**
- _filled = output filled DEM tiles define in yml. DEM tiles, depression filling procedure, first pass: individual tile processing
- _labels = output watershed label raster tiles define in yml. Watershed labels (depression filling procedure)
- _graph = output watershed graph define in yml. Cross-tile watershed connection graph (depression filling procedure)
- _spillover = output watershed spillover define in yml. Tile spillovers, z resolution of dem-watershed-graph (depression filling procedure)
- _resolved = output final filled DEM tiles define in yml. DEM tiles, depression filling procedure, second pass: cross-tile resolution based on spillover graph
- _offset = ?
- _exterior_data = ?

In [14]:
# uncomment below to show default parameters
params.__dict__.items()

dict_items([('_elevations', 'dem'), ('_hydrography', 'stream-network-draped'), ('_filled', 'dem-filled'), ('_labels', 'dem-watershed-labels'), ('_graph', 'dem-watershed-graph'), ('_spillover', 'dem-watershed-spillover'), ('_resolved', 'dem-filled-resolved'), ('_offset', -1.0), ('_exterior_data', 9000.0)])

In [15]:
# change default parameters
params.elevations = 'smoothed'
params.exterior_data = 0.0

In [16]:
# label watershed
for tile in DepressionFill.config.tileset().tiles():
    DepressionFill.LabelWatersheds(row=tile.row, col=tile.col, params=params, overwrite=True)

C:\Users\lmanie01\Documents\Gitlab\python-fct\fct\drainage\DepressionFill.py:162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  z=np.array([
C:\Users\lmanie01\Documents\Gitlab\python-fct\fct\drainage\DepressionFill.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  labels=np.array([


In [17]:
params.labels.tilename(row=1, col=1)

WindowsPath('tutorials/dem_to_dgo/outputs/GLOBAL/DEM/10K/WATERSHED_LABELS/WATERSHED_LABELS_01_01.tif')

In [18]:
params.__dict__.items()

dict_items([('_elevations', 'smoothed'), ('_hydrography', 'stream-network-draped'), ('_filled', 'dem-filled'), ('_labels', 'dem-watershed-labels'), ('_graph', 'dem-watershed-graph'), ('_spillover', 'dem-watershed-spillover'), ('_resolved', 'dem-filled-resolved'), ('_offset', -1.0), ('_exterior_data', 0.0)])